这个是asr的json输出转换脚本，用来验证转换效果

In [1]:
import json

# === 配置 ===
INPUT_JSON = "input.json"
OUTPUT_ASS = "output.ass"
NORMAL_COLOR = "&HFFFFFF&"  # 白色
HIGHLIGHT_COLOR = "&H00FFFF&"  # 黄色
MAX_GAP = 5.0

# === ASS 样式头部 ===
ASS_HEADER = """[Script Info]
ScriptType: v4.00+
Collisions: Normal
PlayResX: 1920
PlayResY: 1080
Timer: 100.0000

[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, BackColour, Bold, Italic, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
Style: Default,Microsoft YaHei,60,&H00FFFFFF,&H000000FF,&H00000000,0,0,1,1,0,2,30,30,30,1

[Events]
Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
"""

def ass_time(t):
    h = int(t // 3600)
    m = int((t % 3600) // 60)
    s = t % 60
    return f"{h}:{m:02d}:{s:05.2f}"

def stylize_word(text, index):
    styled = ""
    for i, ch in enumerate(text):
        if i == index:
            styled += r"{\c" + HIGHLIGHT_COLOR + "}" + ch + r"{\c" + NORMAL_COLOR + "}"
        else:
            styled += ch
    return styled

def process(data):
    events = []

    for i, seg in enumerate(data):
        text = seg["text"]
        words = seg["words"]
        seg_start = seg["start_s"]
        seg_end = seg["end_s"]

        # 每个字分别高亮
        for j in range(len(words)):
            w_start, w_end, _, _ = words[j]
            styled = stylize_word(text, j)
            events.append(f"Dialogue: 0,{ass_time(w_start)},{ass_time(w_end)},Default,,0,0,0,,{styled}")

            # 填补 word gap
            if j + 1 < len(words):
                gap_start = w_end
                gap_end = words[j + 1][0]
                if 0 < gap_end - gap_start < MAX_GAP:
                    events.append(
                        f"Dialogue: 0,{ass_time(gap_start)},{ass_time(gap_end)},Default,,0,0,0,,{text}"
                    )

        # 填补 segment 尾部的空隙（对下一个 segment）
        if i + 1 < len(data):
            next_seg = data[i + 1]
            gap_start = seg_end
            gap_end = next_seg["start_s"]
            if 0 < gap_end - gap_start < MAX_GAP:
                events.append(
                    f"Dialogue: 0,{ass_time(gap_start)},{ass_time(gap_end)},Default,,0,0,0,,{text}"
                )

    return events

def main():
    with open(INPUT_JSON, "r", encoding="utf-8") as f:
        data = json.load(f)

    ass_lines = [ASS_HEADER]
    ass_lines += process(data)

    with open(OUTPUT_ASS, "w", encoding="utf-8") as f:
        f.write("\n".join(ass_lines))

if __name__ == "__main__":
    main()
